In [1]:
import ipywidgets as widgets
# from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np


prbs=pd.Series(['x[7]^x[4]','x[9]^x[5]','x[11]^x[9]','x[13]^x[12]^x[2]^x[1]','x[23]^x[18]','x[31]^x[28]','x[58]^x[39]'],index=[7,9,11,13,23,31,58])
def prbs_on_last(x, prbs_logic):
    x=[0]+x
    return eval(prbs_logic)
def to_string(list_of_bits):
    string_to_print="".join([str(i) for i in list_of_bits])
    string_to_print_bin='_'.join(string_to_print[i:i+8] 
                                 for i in range(0, len(string_to_print), 8))
    string_to_print_hex='_'.join('{:04x}'.format(int(string_to_print[i:i+16],2)) 
                                 for i in range(0, len(string_to_print), 16))
    string_to_print_hex=string_to_print_hex.rsplit('-',1)[0]  # drop the last one that doesnt have 16 bits
        
    return string_to_print_bin,string_to_print_hex

widget_layout=dict(width='200px')
prbs_type=widgets.RadioButtons(options=prbs.index.values, description='prbs type', layout=widget_layout)
dec_seed=widgets.IntText(value=11, description='dec seed', layout=widget_layout)
print_size=widgets.IntText(value=1000, description='print size', layout=widget_layout)
polymon=widgets.Text(description='polymonial', layout=widget_layout)
plot=widgets.ToggleButton(value=False,description='plot',button_style='',tooltip='Description')
pam4=widgets.ToggleButton(value=False,description='pam4',button_style='',tooltip='Description')
bar_plot=widgets.ToggleButton(value=False,description='bar_plot',button_style='',tooltip='Description')
offset=widgets.IntText(value=0,min=0,max=60,step=1,description='offset',continuous_update=False,orientation='horizontal',readout=True,readout_format='d', layout=widget_layout)
    
def p(prbs_type, dec_seed, print_size, polymon, plot, pam4, bar_plot, offset):
    
    # seed:
    if dec_seed<1:
        dec_seed=np.random.randint(1,2**(prbs_type+1),dtype=np.int64)
    bin_seed='{:0{numbers}b}'.format(dec_seed, numbers=prbs_type)
    bin_seed, hex_seed=to_string(bin_seed)
    output=[int(x) for x in list('{0:0{numbers}b}'.format(dec_seed, numbers=prbs_type))]
    if not len(polymon): polymon = prbs[prbs_type]
        
    # running polymon:
    for _ in range(int(2**prbs_type)-1-len(output))[:print_size]:
        output+=[prbs_on_last(output[-prbs_type:],polymon)]
    output=output[offset:print_size]
    # prints
    print('-'*150)
    print('using polynom: {polynom}'.format(polynom=prbs[prbs_type]))
    print('\nseed:\n\tseed dec:\n\t\t{dec}\n\tseed bin:\n\t\t{bin}\n\tseed hex:\n\t\t{hex}\n'.format(dec=dec_seed, bin=bin_seed, hex=hex_seed))

    string_to_print_dec,string_to_print_hex=to_string(output)
    print('\nbin:\n'+string_to_print_dec)
    print('\nhex:\n'+string_to_print_hex)
    if plot:
        import plotly as py
        import cufflinks as cf

        py.offline.init_notebook_mode(connected=True)
        df=pd.DataFrame(output, columns=['nrz'])
        if pam4:
            df['pam4']=df.nrz+df.nrz.shift()*2
            val=df.pam4.iloc[1::2]
            tickvals=[0,1,2,3]
        else:
            val=df.nrz
            tickvals=[0,1]
        if bar_plot:
            fig=val.head(2000).figure(kind='bar', layout=dict(yaxis=dict(tickvals=tickvals)))
        else:
            fig=val.head(2000).figure(mode='markers+lines', size=4, layout=dict(yaxis=dict(tickvals=tickvals)))
            fig['data'][0]['line']['shape']='hvh' # you have hv, vh, hvhv, vhv, spline and linear
        py.offline.iplot(fig)
out = widgets.interactive_output(p, dict(prbs_type=prbs_type, dec_seed=dec_seed, print_size=print_size, polymon=polymon, plot=plot, pam4=pam4, bar_plot=bar_plot, offset=offset))
widgets.VBox([widgets.HBox([widgets.VBox([dec_seed, 
                                          print_size, 
                                          polymon, 
                                          offset]),
                            widgets.VBox([prbs_type])], layout=widgets.Layout(width='450px',border='solid 1px')),
              widgets.HBox([plot, pam4, bar_plot]), 
              widgets.VBox([out])])